In [22]:
import pandas as pd
import geopandas as gpd
import folium
import branca
from geopandas.tools import sjoin
from shapely.geometry import Point

# Load the data into a DataFrame
df = pd.read_csv('swiss_no_outliers_and_cleaned.csv')

# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Address_Longitude, df.Address_Latitude))

# Ensure the GeoDataFrame has a CRS
gdf.crs = "EPSG:4326"

# Load GeoJSON data into a GeoDataFrame
areas = gpd.read_file('GEN_A4_GEMEINDEN_2019_epsg4326.json')

# Perform spatial join
joined = gpd.sjoin(gdf.to_crs(areas.crs), areas, how='inner', op='within')

# Calculate average rent for each area
joined['Rent'] = joined['Rent'].astype(float)  # Ensure 'Rent' is float type
area_avg_rent = joined.groupby('NAME')['Rent'].mean()

# Create a map centered at Zurich
map_zurich = folium.Map(location=[47.3769, 8.5417], zoom_start=12)

# Create a color map
colormap = branca.colormap.LinearColormap(
    colors=['green', 'yellow', 'red'],
    index=area_avg_rent.quantile([0.2, 0.8]),
    vmin=area_avg_rent.min(),
    vmax=area_avg_rent.max()
)

# Add the color map to the map
colormap.add_to(map_zurich)

# Add a GeoJson object to the map
folium.GeoJson(
    areas,
    style_function=lambda feature: {
        'fillColor': colormap(area_avg_rent[feature['properties']['NAME']]),
        'color': 'black',
        'weight': 1,
    }
).add_to(map_zurich)

map_zurich.save("zurich_map.html")


c:\Users\remy\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


KeyError: 'Kilchberg (ZH)'